In [1]:
pip install pybit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 1.6 MB/s 
     |████████████████████████████████| 55 kB 1.9 MB/s 
     |████████████████████████████████| 106 kB 27.1 MB/s 
  Created wheel for pybit: filename=pybit-2.4.1-py2.py3-none-any.whl size=48209 sha256=0d4af1a02431ef798d15857e2fa71e8abde37f7d024daf9c81c76fb4a68df811
  Stored in directory: /root/.cache/pip/wheels/80/ca/f2/63bafd2858cefce6e175b7b2eab77ac90436b910d944248805
Successfully built pybit


In [2]:
pip install ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29104 sha256=0dc24953b2227365a5b418e39b80cbb237c5e2ba132fb7d783c67ab14c4f27e9
  Stored in directory: /root/.cache/pip/wheels/31/31/f1/f2ff471bbc5b84a4b973698ceecdd453ae043971791adc3431
Successfully built ta


In [3]:
from pybit import spot
import pandas as pd
import ta

In [4]:
api_key =''
api_secret = ''

In [ ]:
session = spot.HTTP(endpoint="https://api.bybit.com")

In [ ]:
def klines(symbol):
  kline = session.query_kline(symbol=symbol, interval="1m")
  kline = kline['result']
  df = pd.DataFrame(kline)
  df = df.iloc[:,:6]
  df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
  df = df.set_index('Time')
  df.index = pd.to_datetime(df.index, unit='ms')
  df = df.astype(float)
  return df

def strategy(symbol, qty, open_position = False):

  while True:
    df = klines(symbol)
    if not open_position:
      if ta.trend.macd_diff(df.Close).iloc[-1] > 0 \
      and ta.trend.macd_diff(df.Close).iloc[-2] < 0:
        order = session.place_active_order(symbol=symbol, side='Buy', type='MARKET', qty=qty, timInForce='GTC')
        print(order)
        open_position = True
        break
      
    if open_position:
      while True:
        df = klines(symbol)
        if ta.trend.macd_diff(df.Close).iloc[-1] < 0 \
        and ta.trend.macd_diff(df.Close).iloc[-2] > 0:
          order = session.place_active_order(symbol=symbol, side='Sell', type='MARKET', qty=qty, timInForce='GTC')
          print(order)
          open_position = False
          break


In [ ]:
strategy("WLKNUSDT", 10)